<a href="https://colab.research.google.com/github/codedyasai/Python_MachineLearning/blob/main/21_%EC%9E%90%EC%A0%84%EA%B1%B0_%EC%88%98%EC%9A%94_%EC%98%88%EC%B8%A1_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

# Bike Sharing Demand

- 데이터 셋: https://www.kaggle.com/competitions/bike-sharing-demand/data?select=train.csv
- 알고리즘: 회귀
- 평가지표: RMSLE(캐글에서 요구한 성능 평가 방법, Root Mean Squared Log Error)
- RMSLE의 장점
  - 아웃라이어가 있더라도 크게 영향을 받지 않는다.
  - 상대적 Error를 측정
  - Under Estimation에 큰 페널티를 부여
  - 사이킷런에서는 RMSLE를 제공하지 않아 직접 함수를 만들어서 구현해야 한다.

In [ ]:
df = pd.read_csv('bike_train.csv')
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


## 평가지표(RMSLE), p367

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# log값 변환시 NaN등의 이슈로 log()가 아닌 log1p()를 이용해 RMSLE 계산
def rmsle(y, pred):
  log_y = np.log1p(y)
  log_pred = np.log1p(pred)
  squared_error = (log_y - log_pred) ** 2
  rmsle = np.sqrt(np.mean(squared_error))
  return rmsle

# 사이킷런의 mean_square_error()를 이용해 RMSE계산
def rmse(y, pred):
  return np.sqrt(mean_squared_error(y, pred))

# MSE, RMSE, RMSLE를 모두 계산
def evaluate_regr(y,pred):
  rmsle_val = rmsle(y, pred)
  rmse_val = rmse(y, pred)
  # MAE는 사이킷런의 mean_absolute_error()로 계산
  mae_val = mean_absolute_error(y, pred)
  print('RMSLE: {0:.3f}, RMSE: {1:.3F}, MAE: {2:.3F}'.format(rmsle_val, rmse_val, mae_val))

## **타깃 데이터의 로그변환**

회귀 모델을 적용하기 전에 데이터 세트에 대해서 먼저 처리해야 할 2가지 사항
- 결과값(target)이 정규 분포로 되어 있는가?
- 카테고리형 데이터셋을 One-Hot Encoding을 수행하였는가?

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


**데이터셋 피처**
- datetime: 날짜
- season: 1= 봄, 2= 여름, 3= 가을, 4= 겨울
- holiday: 1= 주말 및 휴일, 0= 평일
- workingday: 1= 주중, 0= 주말 및 휴일
- weather: 1= 맑음/약간흐림, 2= 안개, 3= 가벼운 눈, 5= 심한 눈/비/천둥/번개
- temp: 온도
- atemp: 체감온도
- humidity: 상대 습도
- windspeed: 풍속
- casual: 미등록 사용자 대여수
- registered: 등록된 사용자 대여수
- count: 대여횟수

**datetime**
- 날짜와 시간을 담고 있는 자료형으로 분석을 위해 datetime 형으로 변경이 필요하다.

In [ ]:
df['datetime'] = df.datetime.apply(pd.to_datetime)

# datetime에서 년, 월, 일, 시간 추출
df['year'] = df.datetime.apply(lambda x: x.year)
df['month'] = df.datetime.apply(lambda x: x.month)
df['day'] = df.datetime.apply(lambda x: x.day)
df['hour'] = df.datetime.apply(lambda x: x.hour)

**필요없는 피처 삭제**
- datetime 컬럼을 통해 년,월,일, 시 정보를 획득하였으니 더이상 필요없음
- casual과 registered 두 피처가 더해져서 target에 해당하는 count이기 때문에 필요없음

In [ ]:
drop_col = ['datetime', 'casual', 'registered']
df.drop(drop_col, axis= 1, inplace= True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   holiday     10886 non-null  int64  
 2   workingday  10886 non-null  int64  
 3   weather     10886 non-null  int64  
 4   temp        10886 non-null  float64
 5   atemp       10886 non-null  float64
 6   humidity    10886 non-null  int64  
 7   windspeed   10886 non-null  float64
 8   count       10886 non-null  int64  
 9   year        10886 non-null  int64  
 10  month       10886 non-null  int64  
 11  day         10886 non-null  int64  
 12  hour        10886 non-null  int64  
dtypes: float64(3), int64(10)
memory usage: 1.1 MB


# 모델링

In [ ]:
x = df.drop(['count'], axis=1, inplace= False)
y = df['count']

In [ ]:
## 훈련셋과 테스트셋 분리

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size= 0.2, random_state= 0)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

evaluate_regr(y_test, pred)

RMSLE: 1.191, RMSE: 141.973, MAE: 106.387


- 평가지표를 통해 예측한 결과는 예측 오류를 구하는 값들이 모두 큰 값이다.
- 전처리가 제대로 되지 않아서 이런 문제가 발생했다고 예상할 수 있다.
- 실제값과 예측값의 차이가 어느정도인지 확인이 필요해보인다.

In [ ]:
# 실제값과 예측값의 차이를 확인하기 위해
# 결과 데이터프레임을 생성해서 가장 큰 차이를 보이는 값 5개를 확인

def error_top5(y_test, pred, n_tops= 5):
    result_df = pd.DataFrame(y_test.values, columns = ['Actual'])
    result_df['Pred'] = np.round(pred)
    result_df['diff'] = np.abs(result_df['Actual'] - result_df['Pred'])

    # 예측값과 실제값의 차이가 가장 큰 순서대로 추출
    print(result_df.sort_values('diff', ascending= False)[:n_tops])

In [ ]:
error_top5(y_test, pred, n_tops= 5)

      Actual   Pred   diff
1618     890  321.0  569.0
966      884  325.0  559.0
412      745  193.0  552.0
454      721  177.0  544.0
1003     713  171.0  542.0


- 실제 대여횟수와 예측 대여횟수의 차이가 500회 이상 나는 것은 예측 오류가 크다는 증거이다.
- 큰 예측 오류가 발생한다면 가장 먼저 살펴봐야할 것인 타깃의 분포가 정규 분포를 따르고 있는지를 확인하는 것이다.